In [1]:
import dotenv
import os

dotenv.load_dotenv('env.txt')
domo_username = os.environ.get('DOMO_USERNAME')
domo_instance = os.environ.get('DOMO_INSTANCE')
domo_password = os.environ.get('DOMO_PASSWORD')



domo-community


In [29]:
import requests

def get_full_auth(domo_instance, domo_username, domo_password) -> str: # returns a session token
    """use username and password to generate an access token"""
    
    url = f'https://{domo_instance}.domo.com/api/content/v2/authentication'

    body = {
        'method': 'password',
        'emailAddress': domo_username,
        'password': domo_password
    }

    res = requests.request(method='POST', url=url, json=body)
    data = res.json()
    
    token = data.get('sessionToken')
    if not token:
        raise Exception('unable to retrieve a session token')
        
    return token

session_token = get_full_auth(domo_instance, domo_username, domo_password)
assert session_token

In [35]:
def generate_query_dataflows_body(count = 10 , offset = 0):
    return {"entities":["DATAFLOW"],
            "count":count,"offset":offset,
            "combineResults":False,
            "query":"*",
            "filters":[{"filterType":"term","field":"data_flow_type","value":"MAGIC","name":"Magic ETL v2","not":False}]}
                                
body_dataflows = generate_query_dataflows_body(10,0)
body_dataflows

{'entities': ['DATAFLOW'],
 'count': 10,
 'offset': 0,
 'combineResults': False,
 'query': '*',
 'filters': [{'filterType': 'term',
   'field': 'data_flow_type',
   'value': 'MAGIC',
   'name': 'Magic ETL v2',
   'not': False}]}

In [52]:
def query_datacenter(domo_instance, session_token, body , return_raw: bool= False, debug_api : bool = False):
    """function that queries the datacenter and for a specific body"""
    
    url = f'https://{domo_instance}.domo.com/api/search/v1/query'
    
    if debug_api:
        print({'url' : url,
               'body' : body})
        
    headers = {'x-domo-authentication' : session_token}
    
    res = requests.request(url = url, method= 'POST', headers = headers ,json = body)
    
    if return_raw:
        return res
    
    data = res.json()
    
    return data.get('searchObjects')

dataflows_ls = query_datacenter(domo_instance= domo_instance, session_token = session_token, body =body_dataflows, debug_api = True, return_raw = False)

print(len(dataflows_ls))
dataflows_ls[0]

{'url': 'https://domo-community.domo.com/api/search/v1/query', 'body': {'entities': ['DATAFLOW'], 'count': 10, 'offset': 0, 'combineResults': False, 'query': '*', 'filters': [{'filterType': 'term', 'field': 'data_flow_type', 'value': 'MAGIC', 'name': 'Magic ETL v2', 'not': False}]}}
10


{'entityType': 'dataflow',
 'databaseId': '227',
 'searchId': {'indexName': None,
  'databaseId': '227',
  'customer': 'mmmm-0012-0200',
  'entityType': 'dataflow'},
 'createDate': 1686863564000,
 'lastModified': 1699722570000,
 'lastIndexed': 1699722576782,
 'highlightedFields': {},
 'language': 'English',
 'requestAccess': False,
 'score': 10.000001,
 'name': 'DDX_Landing Page_QA',
 'description': '',
 'ownedById': '1893952720',
 'ownedByName': 'Jae Wilson1',
 'tags': [],
 'lastRunDate': 1686863796000,
 'inputDatasets': [{'name': 'monit_user_accesslist',
   'id': '6ddbcb8d-0f38-48ad-bd73-f6b35c4b7daf'}],
 'outputDatasets': [{'name': 'DDX_Landing Page_QA',
   'id': '51260f1d-b146-4dff-a1d8-68b7aa0b740b'}],
 'runCount': 2,
 'successRate': 1.0,
 'dataFlowType': 'MAGIC',
 'status': 'SUCCESS',
 'deleted': False,
 'passwordProtected': False,
 'abandoned': False,
 'owners': [{'id': '1893952720',
   'type': 'USER',
   'displayName': 'Jae Wilson1'}],
 'ownersLocalized': {'localizedMessage': '

In [50]:
dataflow_ids = [dataflow.get('databaseId') for dataflow in dataflows_ls]
dataflow_ids

['227', '131', '37', '185', '38', '226', '124', '46', '81', '75']

In [56]:
def get_dataflow_definition(dataflow_id, session_token, return_raw: bool = False):
    url = f'https://{domo_instance}.domo.com/api/dataprocessing/v1/dataflows/{dataflow_id}'
    
    headers = {'x-domo-authentication': session_token}
    
    res = requests.request(method = 'get', url = url, headers = headers)
    
    if return_raw:
        return res
    
    data = res.json()
    
    return data

get_dataflow_definition(dataflow_ids[0], session_token = session_token)

{'id': 227,
 'name': 'DDX_Landing Page_QA',
 'dapDataFlowId': '89b76241-30dd-483e-b913-357c5b208db3',
 'responsibleUserId': 1893952720,
 'runState': 'ENABLED',
 'onboardFlowVersion': {'id': 975,
  'timeStamp': 1699722571000,
  'authorId': 1893952720,
  'numInputs': 1,
  'numOutputs': 1,
  'executionCount': 0,
  'executionSuccessCount': 0,
  'versionNumber': 4},
 'lastExecution': {'id': 327613,
  'onboardFlowId': 227,
  'previewRows': 0,
  'dapDataFlowExecutionId': 'ac68b5ac-5f41-4448-aaf6-698971ac45c9',
  'beginTime': 1686863784000,
  'endTime': 1686863796000,
  'lastUpdated': 1686863796000,
  'failed': False,
  'state': 'SUCCESS',
  'dataFlowVersion': 739},
 'created': 1686863564000,
 'modified': 1699722570000,
 'actions': [{'type': 'LoadFromVault',
   'id': 'b9bb0f01-2517-4595-a077-c9778e5ffc48',
   'name': 'monit_user_accesslist',
   'gui': {'x': 159,
    'y': 144,
    'color': 3238043,
    'colorSource': None,
    'sampleJson': None},
   'dataSourceId': '6ddbcb8d-0f38-48ad-bd73-f6b

In [58]:
dataflow_definition_ls = [get_dataflow_definition(dataflow_id, session_token = session_token) for dataflow_id in dataflow_ids]
dataflow_definition_ls[0]

{'id': 227,
 'name': 'DDX_Landing Page_QA',
 'dapDataFlowId': '89b76241-30dd-483e-b913-357c5b208db3',
 'responsibleUserId': 1893952720,
 'runState': 'ENABLED',
 'onboardFlowVersion': {'id': 975,
  'timeStamp': 1699722571000,
  'authorId': 1893952720,
  'numInputs': 1,
  'numOutputs': 1,
  'executionCount': 0,
  'executionSuccessCount': 0,
  'versionNumber': 4},
 'lastExecution': {'id': 327613,
  'onboardFlowId': 227,
  'previewRows': 0,
  'dapDataFlowExecutionId': 'ac68b5ac-5f41-4448-aaf6-698971ac45c9',
  'beginTime': 1686863784000,
  'endTime': 1686863796000,
  'lastUpdated': 1686863796000,
  'failed': False,
  'state': 'SUCCESS',
  'dataFlowVersion': 739},
 'created': 1686863564000,
 'modified': 1699722570000,
 'actions': [{'type': 'LoadFromVault',
   'id': 'b9bb0f01-2517-4595-a077-c9778e5ffc48',
   'name': 'monit_user_accesslist',
   'gui': {'x': 159,
    'y': 144,
    'color': 3238043,
    'colorSource': None,
    'sampleJson': None},
   'dataSourceId': '6ddbcb8d-0f38-48ad-bd73-f6b

In [63]:

def get_action_list(dd_obj):
    return [{'dataflow_id': dd_obj.get('id'),
                 'datafow_name': dd_obj.get('name'),
                 'action_type': action.get('type')} for action in dd_obj.get('actions')]

get_action_list(dd_obj = dataflow_definition_ls[0])

[{'dataflow_id': 227,
  'datafow_name': 'DDX_Landing Page_QA',
  'action_type': 'LoadFromVault'},
 {'dataflow_id': 227,
  'datafow_name': 'DDX_Landing Page_QA',
  'action_type': 'WindowAction'},
 {'dataflow_id': 227,
  'datafow_name': 'DDX_Landing Page_QA',
  'action_type': 'ExpressionEvaluator'},
 {'dataflow_id': 227,
  'datafow_name': 'DDX_Landing Page_QA',
  'action_type': 'Metadata'},
 {'dataflow_id': 227,
  'datafow_name': 'DDX_Landing Page_QA',
  'action_type': 'SelectValues'},
 {'dataflow_id': 227,
  'datafow_name': 'DDX_Landing Page_QA',
  'action_type': 'ExpressionEvaluator'},
 {'dataflow_id': 227,
  'datafow_name': 'DDX_Landing Page_QA',
  'action_type': 'PublishToVault'}]

In [71]:

action_ls = [get_action_list(dd_obj) for dd_obj in dataflow_definition_ls]

flat_action_ls = [ action for a_list in action_ls for action in a_list]
flat_action_ls[0:5]

[{'dataflow_id': 227,
  'datafow_name': 'DDX_Landing Page_QA',
  'action_type': 'LoadFromVault'},
 {'dataflow_id': 227,
  'datafow_name': 'DDX_Landing Page_QA',
  'action_type': 'WindowAction'},
 {'dataflow_id': 227,
  'datafow_name': 'DDX_Landing Page_QA',
  'action_type': 'ExpressionEvaluator'},
 {'dataflow_id': 227,
  'datafow_name': 'DDX_Landing Page_QA',
  'action_type': 'Metadata'},
 {'dataflow_id': 227,
  'datafow_name': 'DDX_Landing Page_QA',
  'action_type': 'SelectValues'}]

In [75]:
import pandas as pd

df = pd.DataFrame(flat_action_ls)
df[0:5]

,dataflow_id,datafow_name,action_type
0,227,DDX_Landing Page_QA,LoadFromVault
1,227,DDX_Landing Page_QA,WindowAction
2,227,DDX_Landing Page_QA,ExpressionEvaluator
3,227,DDX_Landing Page_QA,Metadata
4,227,DDX_Landing Page_QA,SelectValues


In [76]:
df.to_csv('marks_csv.csv')